In [38]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os

In [39]:
path = '../Test_GBDC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: March 31 2013 : (438, 31)
DataFrame name: June 30 2013 : (445, 31)
DataFrame name: September 30 2013 : (468, 41)
DataFrame name: December 31 2013 : (452, 31)
DataFrame name: March 31 2014 : (483, 31)
DataFrame name: June 30 2014 : (522, 27)
DataFrame name: September 30 2014 : (507, 33)
DataFrame name: December 31 2014 : (533, 31)
DataFrame name: March 31 2015 : (550, 31)
DataFrame name: June 30 2015 : (591, 31)
DataFrame name: September 30 2015 : (573, 1701)
DataFrame name: December 31 2015 : (527, 27)
DataFrame name: March 31 2016 : (623, 27)
DataFrame name: June 30 2016 : (586, 27)
DataFrame name: September 30 2016 : (713, 23)
DataFrame name: December 31 2016 : (690, 23)
DataFrame name: March 31 2017 : (717, 23)
DataFrame name: June 30 2017 : (740, 23)
DataFrame name: September 30 2017 : (752, 41)
DataFrame name: December 31 2017 : (785, 41)
DataFrame name: March 31 2018 : (815, 25)
DataFrame name: June 30 2018 : (869, 25)
DataFrame name: September 30 2018 : (904, 24)

In [115]:
def process_table_fun(soi_table_df, process_tables_shapes):
    print(1, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    print(2, 'shape:', soi_table_df.shape)
    # fewer non-null values, effectively dropping rows with more than 100 non-null values.
    for index, row in soi_table_df.iterrows():
        if row.count() > 100:
            soi_table_df = soi_table_df.drop(index)
    print(0, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    print(3, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=0)
    print(4, 'shape:', soi_table_df.shape)
    soi_table_df.dropna()
    print(5, 'shape:', soi_table_df.shape)

    pattern = r'(?i)investment\s*type?|investment'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    soi_table_df = soi_table_df.loc[soi_table_df[matching_rows].index[0]:].reset_index(
        drop=True)

    soi_table_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    print(soi_table_df.columns)

    print(6, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace('—', 0, regex=False)
    print(7, 'shape:', soi_table_df.shape)

    # strip the whole data frame
    soi_table_df = soi_table_df.applymap(
        lambda x: x.strip() if isinstance(x, str) else x)

    process_tables_shapes.append(soi_table_df.shape)

    return soi_table_df

In [116]:
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/process_tables_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
process_tables = {}
process_tables_shape = []
for dataframe in dataframes:
    print(dataframe)
    process_tables[dataframe] = process_table_fun(
        dataframes[dataframe], process_tables_shape)
    print('\n')
    process_tables[dataframe].to_excel(
        writer, sheet_name=dataframe.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

March_31_2013
1 shape: (438, 31)
2 shape: (438, 31)
0 shape: (438, 31)
3 shape: (438, 17)
4 shape: (424, 17)
5 shape: (424, 17)
Index([                nan,        'Investment', 'Spread\n    Above',
                       nan,          'Interest',                 nan,
                'Maturity',         'Principal',                 nan,
                       nan,                'of',                 nan,
                    'Fair',                 nan,                 nan,
                       nan,                 nan],
      dtype='object')
6 shape: (422, 17)
7 shape: (422, 17)


June_30_2013
1 shape: (445, 31)
2 shape: (445, 31)
0 shape: (445, 31)
3 shape: (445, 27)
4 shape: (431, 27)
5 shape: (431, 27)
Index([           nan,   'Investment', 'Spread Above',            nan,
                  nan,     'Interest',            nan,            nan,
           'Maturity',            nan,            nan,    'Principal',
                  nan,            nan,            nan,            nan,